In [ ]:
# Libraries

#PyTorch, of course
import torch
import torch.nn as nn

#We will need torchvision transforms for data augmentation
from torchvision import transforms

### utilities
# tool to print a nice summary of a network, similary to keras' summary
#from torchsummary import summary

# library to do bash-like wildcard expansion
import glob

# others
import numpy as np
import random
from PIL import Image
from IPython.display import display
from tqdm import tqdm_notebook

import torchvision.models as models
import time
import copy

# a little helper function do directly display a Tensor
def display_tensor(t):
  trans = transforms.ToPILImage()
  display(trans(t))
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%cd /kaggle/input/polytech-ds-2019/polytech-ds-2019/
%pwd

In [ ]:
label_name = {i:category for i, category in enumerate(["Bread","Dairy products", "Dessert", "Egg",
                                                       "Fried Food",  "Meat", "Noodles/Pasta", "Rice",
                                                       "Seafood", "Soup", "Vegetable/Fruit"])}

length_train = len(glob.glob('training'+"/*"))


In [ ]:
class trainDataset(torch.utils.data.Dataset):
    
    def __init__(self, img_dir):
        super().__init__()
        self.img_dir = img_dir
        length_train = len(glob.glob(img_dir+"/*"))
        # use glob to get all image names
        self.img_names = [x.split("/")[6] for x in glob.glob(img_dir + "/*")]
        self.labels = [int(img.rsplit("_")[-2]) for img in self.img_names]

        # PyTorch transforms
        self.transform = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                             transforms.ColorJitter(),
                                               transforms.Resize((224, 224)),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                            ])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, i):
        return self._read_img(i)

    def _read_img(self, i):
        img = Image.open(os.path.join(self.img_dir,self.img_names[i]))
        return self.transform(img), self.labels[i]

def display_tensor(t):
    trans = transforms.ToPILImage()
    display(trans(t))


In [ ]:
training = '/kaggle/input/polytech-ds-2019/polytech-ds-2019/training'
validation = '/kaggle/input/polytech-ds-2019/polytech-ds-2019/validation'
evaluation = '/kaggle/input/polytech-ds-2019/polytech-ds-2019/kaggle_evaluation/'
data = [training, validation]

In [ ]:
def display_tensor(t):
    trans = transforms.ToPILImage()
    display(trans(t))

In [ ]:
ind = 4
train_set = trainDataset(training)
val_set = trainDataset(validation)
img, label = train_set[ind]
print("label of {} (picture #{}) is {}".format(train_set.img_names[ind], ind, label_name[label]))
display_tensor(img)

In [ ]:
img, label = val_set[ind]
print("label of {} (picture #{}) is {}".format(val_set.img_names[ind], ind, label_name[label]))
display_tensor(img)

In [ ]:
# Data loader for the train dataset
data_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

In [ ]:
# Prepare model
data_dir = "./"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnext"

# Number of classes in the dataset
num_classes = 11

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 35

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299
        
    elif model_name == "resnext":
        model_ft = torch.hub.load('pytorch/vision:v0.4.2', 'resnext50_32x4d', pretrained=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Initializing Datasets and Dataloaders...")

In [ ]:

image_datasets ={'train':train_set,'val':val_set}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
#dataloader_eval = torch.utils.data.DataLoader(ev_set, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
# Send the model to GPU
#model_ft = model_ft.to(device)
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
model_ft = model_ft.to(device)
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
class loadEvaluation(torch.utils.data.Dataset):
    
    def __init__(self, img_dir):
        super().__init__()
        self.img_dir = img_dir
        length_train = len(glob.glob(img_dir+"/*"))
        # use glob to get all image names
        self.img_names = [x.split("/")[6] for x in glob.glob(img_dir + "/*")]

        # PyTorch transforms
        self.transform = transforms.Compose([transforms.Resize(input_size),
                                             transforms.CenterCrop(input_size),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                            ])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, i):
        return self._read_img(i)

    def _read_img(self, i):
        img = Image.open(os.path.join(self.img_dir,self.img_names[i]))
        return self.transform(img)

def display_tensor(t):
    trans = transforms.ToPILImage()
    display(trans(t))


In [ ]:
# Load evaluation data
eval_data = loadEvaluation(evaluation)
# Prepare data loader
eval_loader = torch.utils.data.DataLoader(eval_data, batch_size=batch_size, shuffle=True, num_workers=4)


In [ ]:
image = train_set[32][0]
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = image.cuda()

In [ ]:
image = eval_data[42]
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = image.cuda()

In [ ]:
out = model_ft(image)
_, preds = torch.max(out, 1)
preds

In [ ]:
results = []
for tensor in eval_data:
    image=tensor.reshape((1, tensor.shape[0], tensor.shape[1], tensor.shape[2]))
    image = image.cuda()
    out = model_ft(image)
    _, preds = torch.max(out, 1)
    val = preds.cpu().numpy()
    results.append(val[0])

In [ ]:
results=np.array(results)
res=[]
for result in results:
    aux = " "+str(result)
    res.append(aux)


In [ ]:
names = np.array(eval_data.img_names)
names

In [ ]:
names_f = []
for name in names:
    a=name.replace('.jpg', '')
    names_f.append(a)

In [ ]:
names_final = np.array(names_f)


In [ ]:
 df = pd.DataFrame({'Id':names_final, 'Category':res})

In [ ]:
df.to_csv(path_or_buf='/kaggle/working/submission_1.csv', index=False)

In [ ]:
%%bash 
cd /kaggle/working/
less submission.csv